<a href="https://colab.research.google.com/github/MinghaoHan/5223Go/blob/main/ResNet_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this file, we focus on the training and evaluation for the ResNet-50 with and without pretrained weights, the model performance without pretrain weights will be considered as a baseline.

Before running this code with pretrained weights, please make sure you have downloaded the pretrained weights from https://download.pytorch.org/models/resnet50-11ad3fa6.pth

In [ ]:
import os

In [ ]:
# switch to the path where the dataset is stored
%cd /content/drive/MyDrive/CS4243Project/dataset/
# copy the dataset (.zip) to the /content path
!cp frames-TVT.zip /content
# switch to content directory
%cd /content
# unzip dataset zip file
!unzip frames-TVT.zip -d '/content/'

In [ ]:
# check
def count_files_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        file_count = len(files)
        print(f"{root} contains {file_count} files")

dir_path = '/content/frames-TVT/'
count_files_in_folder(dir_path)

/content/frames-TVT/ contains 1 files
/content/frames-TVT/train contains 1 files
/content/frames-TVT/train/weap contains 1183 files
/content/frames-TVT/train/norm contains 1648 files
/content/frames-TVT/val contains 1 files
/content/frames-TVT/val/weap contains 296 files
/content/frames-TVT/val/norm contains 412 files
/content/frames-TVT/test contains 1 files
/content/frames-TVT/test/weap contains 779 files
/content/frames-TVT/test/norm contains 1292 files
/content/frames-TVT/train_all contains 1 files
/content/frames-TVT/train_all/weap contains 1478 files
/content/frames-TVT/train_all/norm contains 2059 files


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 16 07:09:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
print(sys.path)
sys.path.append('/content/drive/MyDrive/CS4243Project/')
sys.path.append('/content/drive/MyDrive/CS4243Project/ResNet50/')

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
import os
import sys
import json

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from tqdm import tqdm
from model import resnet50

import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
if __name__ == '__main__':
    main()

using cuda:0 device.
Using 2 dataloader workers every process
using 3537 images for training, 2071 images for validation.
valid epoch[1/20]: 100%|██████████| 65/65 [01:56<00:00,  1.80s/it]
[epoch 1] train_loss: 0.647  val_accuracy: 0.730  val_f1_score: 0.550
[[1170  122]
 [ 437  342]]
valid epoch[2/20]: 100%|██████████| 65/65 [01:58<00:00,  1.82s/it]
[epoch 2] train_loss: 0.571  val_accuracy: 0.748  val_f1_score: 0.585
[[1181  111]
 [ 411  368]]
valid epoch[3/20]: 100%|██████████| 65/65 [01:59<00:00,  1.84s/it]
[epoch 3] train_loss: 0.523  val_accuracy: 0.750  val_f1_score: 0.598
[[1170  122]
 [ 395  384]]
valid epoch[4/20]: 100%|██████████| 65/65 [01:58<00:00,  1.82s/it]
[epoch 4] train_loss: 0.488  val_accuracy: 0.755  val_f1_score: 0.609
[[1167  125]
 [ 383  396]]
valid epoch[5/20]: 100%|██████████| 65/65 [02:02<00:00,  1.88s/it]
[epoch 5] train_loss: 0.457  val_accuracy: 0.754  val_f1_score: 0.619
[[1148  144]
 [ 365  414]]
valid epoch[6/20]: 100%|██████████| 65/65 [02:02<00:00,  1

KeyboardInterrupt: ignored

In [ ]:
def evaluate_on_test():

    batch_size = 32

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    print("using {} device.".format(device))

    results_file = "/content/drive/MyDrive/CS4243Project/ResNet50/save_weights/test_results{}.txt".format(datetime.datetime.now().strftime("%Y%m%d-%H%M"))

    data_transform = {
        "test": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

    image_path = "/content/frames-TVT/"
    assert os.path.exists(image_path), "{} path does not exist.".format(image_path)
    test_dataset = datasets.ImageFolder(root=os.path.join(image_path, "test"), transform=data_transform["test"])
    test_num = len(test_dataset)

    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    print('Using {} dataloader workers every process'.format(nw))

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=nw)
    print("using {} images for testing.".format(test_num))

    net = resnet50()
    # change fc layer structure
    in_channel = net.fc.in_features
    net.fc = nn.Linear(in_channel, 2) # number of classes

    # load best weights
    model_weight_path = "/content/drive/MyDrive/CS4243Project/ResNet50/save_weights/reset50-9.pth"
    assert os.path.exists(model_weight_path), "file {} does not exist.".format(model_weight_path)
    net.load_state_dict(torch.load(model_weight_path, map_location='cpu'))

    net.to(device)

    # define loss function
    loss_function = nn.CrossEntropyLoss()

    best_acc = 0.0

    all_test_real = []
    all_test_pred = []

    # test
    net.eval()
    acc = 0.0  # accumulate accurate number / epoch
    test_loss = 0.0 # loss
    with torch.no_grad():
        test_bar = tqdm(test_loader, file=sys.stdout)
        for test_data in test_bar:
            test_images, test_labels = test_data
            outputs = net(test_images.to(device))
            loss = loss_function(outputs, test_labels.to(device)) # loss
            test_loss += loss.item() # loss
            predict_y = torch.max(outputs, dim=1)[1]
            all_test_pred.extend(predict_y.cpu().tolist())
            all_test_real.extend(test_labels.tolist())
            acc += torch.eq(predict_y, test_labels.to(device)).sum().item()

            test_bar.desc = "Test epoch[{}/{}]".format(1,1)

    test_accurate = acc / test_num
    conf_mat = confusion_matrix(all_test_real, all_test_pred)
    test_f1 = f1_score(all_test_real, all_test_pred)
    report = classification_report(all_test_real, all_test_pred)

    #print('Checking accuracy:', val_accurate, accuracy_score(all_valid_real, all_valid_pred))
    print('[Tesing]  test_loss: %.3f  test_accuracy: %.3f  test_f1_score: %.3f' %
          (test_loss, test_accurate, test_f1))
    print(conf_mat)
    print(report)

    # write into txt
    # with open(results_file, "a") as f:
    #     txt = "Epoch: {}, valid_acc: {}, valid_f1: {}.".format(epoch, val_accurate, valid_f1)
    #     txt = "train_loss: {}".format('  '.join([f"{i:.4f}" for i in train_loss_epoch]))
    #     txt = "valid_loss: {}".format('  '.join([f"{i:.4f}" for i in valid_loss_epoch]))
    #     f.write(txt + "\n")

    print('Finished Testing')



In [ ]:
def main():

    batch_size = 32
    epochs = 20
    lr = 1e-4

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    print("using {} device.".format(device))

    results_file = "/content/drive/MyDrive/CS4243Project/ResNet50/save_weights/results{}.txt".format(datetime.datetime.now().strftime("%Y%m%d-%H%M"))

    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "val": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

    image_path = "/content/frames-TVT/"
    assert os.path.exists(image_path), "{} path does not exist.".format(image_path)
    train_dataset = datasets.ImageFolder(root=os.path.join(image_path, "train_all"), transform=data_transform["train"])
    train_num = len(train_dataset)

    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=nw)
    # val set
    validate_dataset = datasets.ImageFolder(root=os.path.join(image_path, "test"), transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size, shuffle=False, num_workers=nw)

    print("using {} images for training, {} images for validation.".format(train_num, val_num))

    net = resnet50()

    model_weight_path = "/content/drive/MyDrive/CS4243Project/resnet50-pretrained.pth"
    assert os.path.exists(model_weight_path), "file {} does not exist.".format(model_weight_path)
    net.load_state_dict(torch.load(model_weight_path, map_location='cpu'))

    # freeze feature extractor layer weights
    for param in net.parameters():
        param.requires_grad = False # false: freeze

    # # change fc layer structure
    in_channel = net.fc.in_features
    net.fc = nn.Linear(in_channel, 2) # number of classes
    net.to(device)

    # loss function
    loss_function = nn.CrossEntropyLoss()

    # optimizer
    params = [p for p in net.parameters() if p.requires_grad]
    optimizer = optim.Adam(params, lr=lr)

    best_acc = 0.0

    # loss
    train_loss_epoch = []
    valid_loss_epoch = []

    train_steps = len(train_loader)
    for epoch in range(epochs):
        all_valid_real = []
        all_valid_pred = []
        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            logits = net(images.to(device))
            loss = loss_function(logits, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,epochs,loss)
        train_loss_epoch.append(running_loss) # loss

        # eval
        net.eval()
        acc = 0.0  # accumulate accurate number / epoch
        val_loss = 0.0 # loss
        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                loss = loss_function(outputs, val_labels.to(device)) # loss
                val_loss += loss.item() # loss
                predict_y = torch.max(outputs, dim=1)[1]
                all_valid_pred.extend(predict_y.cpu().tolist())
                all_valid_real.extend(val_labels.tolist())
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

                val_bar.desc = "valid epoch[{}/{}]".format(epoch + 1,epochs)

        valid_loss_epoch.append(val_loss) # loss
        val_accurate = acc / val_num
        conf_mat = confusion_matrix(all_valid_real, all_valid_pred)
        valid_f1 = f1_score(all_valid_real, all_valid_pred)

        #print('Checking accuracy:', val_accurate, accuracy_score(all_valid_real, all_valid_pred))
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f  val_f1_score: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate, valid_f1))
        print(conf_mat)

        # write into txt
        with open(results_file, "a") as f:
            # validation metrics
            txt = "Epoch: {}, valid_acc: {}, valid_f1: {}.".format(epoch, val_accurate, valid_f1)
            f.write(txt + "\n")
            txt = "train_loss: {}".format('  '.join([f"{i:.4f}" for i in train_loss_epoch]))
            f.write(txt + "\n")
            txt = "valid_loss: {}".format('  '.join([f"{i:.4f}" for i in valid_loss_epoch]))
            f.write(txt + "\n")


        # if val_accurate > best_acc:
        #     best_acc = val_accurate
        torch.save(net.state_dict(), "/content/drive/MyDrive/CS4243Project/ResNet50/save_weights/reset50-{}.pth".format(epoch))

    # write into txt
    with open(results_file, "a") as f:
        # train loss & valid loss
        txt = "train_loss: {}".format('  '.join([f"{i:.4f}" for i in train_loss_epoch]))
        f.write(txt + "\n")
        txt = "valid_loss: {}".format('  '.join([f"{i:.4f}" for i in valid_loss_epoch]))
        f.write(txt + "\n")

    print('Finished Training')

    plt.plot([*range(epochs)], train_loss_epoch, label='Train Loss')
    plt.plot([*range(epochs)], valid_loss_epoch, label='Validation Loss')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Train Loss vs. Validation Loss')
    plt.savefig('/content/drive/MyDrive/CS4243Project/ResNet50/Loss.png')
    plt.show()

In [ ]:
evaluate_on_test()

using cuda:0 device.
Using 2 dataloader workers every process
using 2071 images for testing.
Test epoch[1/1]: 100%|██████████| 65/65 [02:00<00:00,  1.85s/it]
[Tesing]  test_loss: 31.649  test_accuracy: 0.764  test_f1_score: 0.639
[[1152  140]
 [ 348  431]]
              precision    recall  f1-score   support

           0       0.77      0.89      0.83      1292
           1       0.75      0.55      0.64       779

    accuracy                           0.76      2071
   macro avg       0.76      0.72      0.73      2071
weighted avg       0.76      0.76      0.75      2071

Finished Testing
